<a href="https://colab.research.google.com/github/HelloMrAman/my_repo/blob/main/myFinances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Personal Finances



###Data cleaning and Preperation



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_excel('/content/sample_data/my_finance_data.xlsx')

print(data.head())
print(data.info())

In [ ]:
print(data.isnull().sum())

### Data Analysis

In [ ]:
data.describe()

In [ ]:
print(data['Category'].value_counts())

*Spending by category*

In [ ]:
data['Category'].value_counts().plot(kind='bar')
plt.title('Spending by Category')
plt.show()

*Spending by month*

In [ ]:
# First we will separate Income/Expense column
data['Income'] = data['Debit/Credit'] * (data['Income/Expense'] == 'Income')
data['Expense'] = data['Debit/Credit'] * (data['Income/Expense'] == 'Expense') * -1
data

In [ ]:
# Separate columns for Debit and Credit

data['Debit'] = data['Debit/Credit'] * (data['Debit/Credit'] < 0)
data['Credit'] = data['Debit/Credit'] * (data['Debit/Credit'] >= 0)
data

In [ ]:
expenses = data[data['Income/Expense'] == 'Expense']

# Try converting 'Date' to DateTime format
try:
  expenses = expenses.assign(Date=pd.to_datetime(expenses['Date'], format='%y-%m-%d'))
except pd.errors.ParserError:
  pass

expenses = expenses.assign(Month=expenses['Date'].dt.month_name())

# Group by month and calculate total expenses
monthly_spending = expenses.groupby('Month')['Debit'].sum() * -1

monthly_spending.plot(kind='bar', color='red')
plt.title('Monthly Spending')
plt.xlabel('Month')
plt.ylabel('Total Expense')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


*Monthly earning*

In [ ]:
incomes = data[data['Income/Expense'] == 'Income']

# Try converting 'Date' to DateTime format
try:
  incomes = incomes.assign(Date=pd.to_datetime(incomes['Date'], format='%y-%m-%d'))
except pd.errors.ParserError:
  pass

incomes = incomes.assign(Month=incomes['Date'].dt.month_name())

# Group by month and calculate total incomes
monthly_Earning = incomes.groupby('Month')['Credit'].sum()

monthly_Earning.plot(kind='bar', color='green')
plt.title('Monthly Income')
plt.xlabel('Month')
plt.ylabel('Total Income')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

*Income Vs Expenses Over time*

Area Bar Chart

In [ ]:
incomes = data[data['Income/Expense'] == 'Income']
expenses = data[data['Income/Expense'] == 'Expense']

# Ensure 'Date' is formatted correctly
incomes.loc[:, 'Date'] = pd.to_datetime(incomes['Date'], format='%y-%m-%d')
expenses.loc[:, 'Date'] = pd.to_datetime(expenses['Date'], format='%y-%m-%d')

# Set 'Date' as index if it represents time series data
incomes.set_index('Date', inplace=True)
expenses.set_index('Date', inplace=True)

# Resample to monthly data
incomes_resampled = incomes.resample('M')['Credit'].sum()
expenses_resampled = expenses.resample('M')['Debit'].sum() * -1

# Adjust the timeframe as needed (e.g., incomes_resampled.loc['2023-01':'2023-12'])
year_income = incomes_resampled.loc[incomes_resampled.index.year == incomes_resampled.index[-1].year]
year_expense = expenses_resampled.loc[expenses_resampled.index.year == expenses_resampled.index[-1].year]

# area chart
fig, ax = plt.subplots()
year_income.plot(kind='area', alpha=0.4, color='green', label='Income', ax=ax)
year_expense.plot(kind='area', alpha=0.4, color='red', label='Expense', ax=ax)

# Customize the plot title and labels based on your data
plt.title('Income vs Expenses')
plt.xlabel('Month')
plt.ylabel('Amount')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



### Budgeting and Goal setting

*Saving Rate*

In [ ]:
# Series to DataFrame with a single column (to use merge)
monthly_income_df = incomes_resampled.to_frame('Credit')
monthly_expense_df = expenses_resampled.to_frame('Debit')

# monthly savings
monthly_data = monthly_income_df.merge(monthly_expense_df, how='left', on='Date')
monthly_data['Savings'] = monthly_data['Credit'] - monthly_data['Debit']

# total income, expenses, and savings
total_income = monthly_data['Credit'].sum()
total_expenses = monthly_data['Debit'].sum()
total_savings = monthly_data['Savings'].sum()

# savings rate (assuming total income is not zero)
if total_income != 0:
    savings_rate = (total_savings / total_income) * 100
else:
    savings_rate = 0  # Handle zero income case (division by zero)


# Print formatted results
print(f"Total Income: ₹{total_income:.2f}")
print(f"Total Expenses: ₹{total_expenses:.2f}")
print(f"Total Savings: ₹{monthly_data['Savings'].sum():.2f}")
print(f"Savings Rate: {savings_rate:.2f}%")


*Setting Financial Goal*

-->Define Goal (eg. saving for vacation costing ₹1,00,000)

-->Setting a target for achieving the goal (e.g., 6 months from now).

In [ ]:
# Calculating saving needed
goal_amnt = 100000
target_months = 6

# monthly savings needed to reach the goal
monthly_savings_needed = goal_amnt/target_months

In [ ]:
monthly_data = monthly_income_df.merge(monthly_expense_df, how='left', on='Date')
monthly_data = monthly_data.reset_index()

monthly_data['Savings'] = monthly_data['Credit'] - monthly_data['Debit']

# Ploting monthly savings over time
plt.figure(figsize=(10, 6))  # Adjust figure size for readability
plt.plot(monthly_data['Date'], monthly_data['Savings'], marker='o', color='blue', label='Actual Savings')
plt.axhline(y=monthly_savings_needed, color='red', linestyle='--', label=f'Target Savings: ₹{monthly_savings_needed:.2f}')
plt.title('Savings Progress Over Time')
plt.xlabel('Date')
plt.ylabel('Savings Amount')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()